OR Schedule Adherence Analysis

In [22]:
import pandas as pd
import pandasql as ps
from pandasql import sqldf
# Define a reusable function for running SQL queries
run_query = lambda query: sqldf(query, globals())

In [62]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [23]:
# Import the data (housed here: https://data.world/drjfalk/operating-room-utilization)
or_df = pd.read_csv('redacted')

In [24]:
# Convert the timestamps to datetime objects 
# Note: I would normally do this with SQL date functions, but Pandas uses SQLLite which has clunky date formatting
or_df[['OR Schedule', 'Wheels In', 'Start Time', 'End Time', 'Wheels Out']] = or_df[['OR Schedule', 'Wheels In', 'Start Time', 'End Time', 'Wheels Out']].apply(pd.to_datetime, format='%m/%d/%y %I:%M %p')
or_df['Date'] = pd.to_datetime(or_df['Date'], format='%m/%d/%y')

In [25]:
# Rename and format the columns to be more sql friendly. Look at the first few rows to familizarize myself with the data.
or_clean = sqldf("""
    select
        "Encounter ID" as encounter_id
        , "Date" as encounter_date
        , "OR Suite" as or_suite
        , "Service" as service
        , "CPT Code" as cpt_code
        , "CPT Description" as cpt_description
        , "Booked Time (min)" as booked_time_min
        , "OR Schedule" as or_schedule
        , "Wheels In" as wheels_in
        , "Start Time" as start_time
        , "End Time" as end_time
        , "Wheels Out" as wheels_out          
    from or_df
""")
print(or_clean.head())

   encounter_id              encounter_date  or_suite      service  cpt_code  \
0         10001  2022-01-03 00:00:00.000000         1     Podiatry     28110   
1         10002  2022-01-03 00:00:00.000000         1     Podiatry     28055   
2         10003  2022-01-03 00:00:00.000000         1     Podiatry     28297   
3         10004  2022-01-03 00:00:00.000000         1     Podiatry     28296   
4         10005  2022-01-03 00:00:00.000000         2  Orthopedics     27445   

                             cpt_description  booked_time_min  \
0   Partial ostectomy, fifth metatarsal head               90   
1  Neurectomy, intrinsic musculature of foot               60   
2                       Lapidus bunionectomy              150   
3         Bunionectomy with distal osteotomy              120   
4        Arthroplasty, knee, hinge prothesis              120   

                  or_schedule                   wheels_in  \
0  2022-01-03 07:00:00.000000  2022-01-03 07:05:00.000000   
1  202

In [107]:
# Create a new sqldf that calculates the fields needed for this analysis.
# This will remain at the encounter level, with various aggregations later on.

or_intervals = sqldf("""
    with
        intervals as (
            select *
                     
                -- Add the day of week in case that makes a difference in the analysis 
                , strftime('%w',encounter_date) as day_of_week     
                     
                -- Add the scheduled end time by adding the booked_time_min to the or_schedule      
                , strftime('%Y-%m-%d %H:%M:%S', or_schedule, '+' || booked_time_min || ' minutes') as scheduled_end_time
                     
                -- Calculate the time between each event to explore whether there is wasted time between events 
                , (strftime('%s', wheels_in) - strftime('%s', or_schedule))/60 as scheduled_to_wheels_in_min
                , (strftime('%s', start_time) - strftime('%s', or_schedule))/60 as or_schedule_to_start_min
                , (strftime('%s', start_time) - strftime('%s', wheels_in))/60 as wheels_in_to_start_min
                , (strftime('%s', end_time) - strftime('%s', start_time))/60 as start_to_end_min
                , (strftime('%s', wheels_out) - strftime('%s', end_time))/60 as end_to_wheels_out_min
                , (strftime('%s', wheels_out) - strftime('%s', wheels_in))/60 as wheels_in_to_wheels_out_min
                , (strftime('%s', wheels_out) - strftime('%s', or_schedule))/60 as scheduled_to_wheels_out_min 

                -- Add a row_number to determine what order the procedures happened in. Partitioned by suite and
                -- encounter date since each day the order should reset with 1 surgery per OR.      
                , row_number() over(partition by or_suite, encounter_date order by start_time) as surgery_order_by_day
                     
                -- Add the prior and next encounter in the same suite on the same date 
                , lead(encounter_id) over(partition by or_suite, encounter_date order by start_time) as next_encounter_id
                , lag(encounter_id) over(partition by or_suite, encounter_date order by start_time) as prior_encounter_id 

            from or_clean
        )

        , late_time as (
            select *
                , (strftime('%s', wheels_out) - strftime('%s', scheduled_end_time))/60 as minutes_over_schedule
            from intervals 
        )             

    -- Add the late indicators and do a self join to get the next encounter's data 
        select 
            current_encounter.*         
            -- Create a boolean flag to indicate whether the wheels out was more than 5 minutes after the scheduled end time 
            , case when current_encounter.scheduled_to_wheels_in_min > 5 then 1 else 0 end as started_late         
            , case when current_encounter.minutes_over_schedule > 5 then 1 else 0 end as ended_late

            -- Add the relevant fields from the next encounter                  
            , next_encounter.or_suite as next_or_suite        
            , next_encounter.service as next_service
            , next_encounter.cpt_code as next_cpt_code
            , next_encounter.cpt_description as next_cpt_description 
            , next_encounter.booked_time_min as next_booked_time_min
            , next_encounter.or_schedule as next_or_schedule
            , next_encounter.wheels_in as next_wheels_in
            , next_encounter.start_time as next_start_time
            , next_encounter.end_time as next_end_time
            , next_encounter.wheels_out as next_wheels_out
            , next_encounter.minutes_over_schedule as next_minutes_over_schedule
            
            -- Same booleans as the current encounter 
            , case when next_encounter.scheduled_to_wheels_in_min > 5 then 1 else 0 end as next_started_late         
            , case when next_encounter.minutes_over_schedule > 5 then 1 else 0 end as next_ended_late
                     

            -- Turn around time between current and next 
            , (strftime('%s', next_encounter.wheels_in) - strftime('%s', current_encounter.wheels_out))/60 as turnaround_time_min
            , (strftime('%s', next_encounter.or_schedule) - strftime('%s', current_encounter.wheels_out))/60 as time_to_next_scheduled_case_min         
                     
        
            -- Based on the booked time and or schedule, we can see the time between cases 
            , (strftime('%s', next_encounter.or_schedule) - strftime('%s', current_encounter.scheduled_end_time))/60 as scheduled_min_to_next_encounter               
        from late_time as current_encounter
        left join late_time as next_encounter
            on current_encounter.next_encounter_id = next_encounter.encounter_id           
""")


In [148]:
print(sqldf("""
    select 
        service
        , avg(wheels_in_to_wheels_out_min) as avg_surgery_time
    from or_intervals
group by 1
    order by  1
   """))

         service  avg_surgery_time
0            ENT         69.096447
1        General        113.000000
2          OBGYN         91.750000
3  Ophthalmology         35.871257
4    Orthopedics        100.959502
5     Pediatrics         66.000000
6        Plastic        103.420290
7       Podiatry         94.329268
8        Urology         70.756477
9       Vascular         81.179191


In [ ]:
print(sqldf("""
    select 
        cpt_description
        , avg(wheels_in_to_wheels_out_min) as avg_surgery_time
    from or_intervals
    where service = 'Ophthalmology'
    group by 1
   """))

                  cpt_description  avg_surgery_time
0  Extracapsular cataract removal         35.871257


In [149]:
print(sqldf("""
        select
            count(*)
            , sum(started_late) as started_late
            , sum(ended_late) as ended_late
            , sum(case when started_late = 1 and ended_late = 1 then 1 else 0 end) as both_late
        from or_intervals
        """))

   count(*)  started_late  ended_late  both_late
0      2172          1628        1636       1534


In [181]:
# How many surgeries ended > 5 min late based on Service?
print(sqldf("""
        select
            service
            , sum(ended_late) as ended_late
            , count(*) as total_surgeries
            , avg(minutes_over_schedule) as avg_late_time
        from or_intervals
        group by 1
        """))

         service  ended_late  total_surgeries  avg_late_time
0            ENT         174              197      31.619289
1        General         117              117      36.666667
2          OBGYN         123              164      16.250000
3  Ophthalmology           0              334     -12.443114
4    Orthopedics         321              321      74.526480
5     Pediatrics         220              220      52.200000
6        Plastic          69              207       9.227053
7       Podiatry         246              246      54.621951
8        Urology         193              193      60.424870
9       Vascular         173              173      64.751445


In [159]:
# Late Time by OR
print(sqldf("""
        select
            or_suite
            , avg(minutes_over_schedule) as avg_min_over_schedule
            , count(*) as num_encounters
        from or_intervals
        --where ended_late = 1
        group by 1
        """))

   or_suite  avg_min_over_schedule  num_encounters
0         1              54.621951             246
1         2              81.059524             252
2         3               3.018223             439
3         4              32.130597             268
4         5              41.765734             286
5         6               9.227053             207
6         7              59.739583             288
7         8              41.860215             186


In [67]:
# How many first surgeries of the day ended?
print(sqldf("""
        select
            ended_late
            , count(*)
        from or_intervals
       where surgery_order_by_day = 1
        group by 1
        """))

   ended_late  count(*)
0           0       167
1           1       329


In [160]:
# Of surgeries with another surgery scheduled after, how many ended late?
print(sqldf("""
        select
            ended_late
            , count(*)
        from or_intervals
        where next_encounter_id is not null
        group by 1
        """))

   ended_late  count(*)
0           0       456
1           1      1220


In [161]:
# Of surgeries with another surgery scheduled after that ended late, how many had the next surgery start, end, or both late?
print(sqldf("""
        select
            sum(next_started_late) as next_started_late
            , sum(next_ended_late) as next_ended_late
            , sum(case when next_started_late = 1 and next_ended_late = 1 then 1 else 0 end) as next_both_late
        from or_intervals
       where next_encounter_id is not null
            and ended_late = 1

        """))

   next_started_late  next_ended_late  next_both_late
0               1220             1220            1220


In [162]:
# Excluding Ophthalmology, how many surgeries started/ended late by order of the day?
print(sqldf("""
        select
            surgery_order_by_day
            , sum(started_late) as started_late
            , sum(ended_late) as ended_late
            , sum(case when started_late = 1 and ended_late = 1 then 1 else 0 end) as both_late
            , count(*) as num_encounters
        from or_intervals
        where service != 'Ophthalmology'
        group by 1
        """))

   surgery_order_by_day  started_late  ended_late  both_late  num_encounters
0                     1           266         329        227             455
1                     2           455         416        416             455
2                     3           414         398        398             435
3                     4           334         334        334             334
4                     5           159         159        159             159


In [185]:
# What time do surgeries end compared to scheduled end of day?

print(sqldf("""
            with end_of_day as (
                        select 
                        encounter_date
                        , or_suite
                        , max(scheduled_end_time) as scheduled_end_of_day
                        , max(wheels_out) as actual_end_of_day
                        from or_intervals
                        where service != 'Ophthalmology'
                        group by 1, 2
            )
        select 
            or_suite
            , avg((strftime('%s', actual_end_of_day) - strftime('%s', scheduled_end_of_day))/60) as avg_end_of_day_diff
        from end_of_day
        group by 1
            """))

print(sqldf("""
            with end_of_day as (
                        select 
                        encounter_date
                        , or_suite
                        , max(scheduled_end_time) as scheduled_end_of_day
                        , max(wheels_out) as actual_end_of_day
                        from or_intervals
                        where service != 'Ophthalmology'
                        group by 1, 2
            )
        select 
             avg((strftime('%s', actual_end_of_day) - strftime('%s', scheduled_end_of_day))/60) as avg_end_of_day_diff
        from end_of_day
            """))

   or_suite  avg_end_of_day_diff
0         1            59.967742
1         2           137.258065
2         3            88.000000
3         4            56.806452
4         5            80.822581
5         6            23.016129
6         7           109.532258
7         8            64.096774
   avg_end_of_day_diff
0            76.485714


In [165]:
# What is the distribution of time surgeries take compared to scheduled time?
print(sqldf("""
            with bins as (
                        select 
                            encounter_id
                            , booked_time_min - wheels_in_to_wheels_out_min
                            , case  
                                when booked_time_min - wheels_in_to_wheels_out_min < -10 then 'Over 10 min late'
                                when booked_time_min - wheels_in_to_wheels_out_min < -5 then '5-10 min late'
                                when booked_time_min - wheels_in_to_wheels_out_min < 0 then '0-5 min late'
                                when booked_time_min - wheels_in_to_wheels_out_min = 0 then 'On time'
                                when booked_time_min - wheels_in_to_wheels_out_min < 5 then '0-5 min early'
                                when booked_time_min - wheels_in_to_wheels_out_min <= 10 then '5-10 min early'
                                when booked_time_min - wheels_in_to_wheels_out_min > 10 then 'Over 10 min early'
                            end as actual_vs_booked
                        from or_intervals
                        where service != 'Ophthalmology'  
            )
        select 
            actual_vs_booked
            , count(*) as num_surgeries
        from bins
        group by 1
            """))



    actual_vs_booked  num_surgeries
0      0-5 min early            145
1       0-5 min late            269
2     5-10 min early            307
3      5-10 min late            297
4            On time             11
5  Over 10 min early            138
6   Over 10 min late            671


In [ ]:
# How does the distribution change if we modify the 6 cpts with the highest discrepancy between booked and actual time?
print(sqldf("""
            with 
                adj_booked_times as (
                    select
                        encounter_id
                        , case
                            when cpt_code = 28110 then 150
                            when cpt_code = 28055 then 90
                            when cpt_code = 28285 then 90
                            when cpt_code = 27445 then 150
                            when cpt_code = 27130 then 150
                            when cpt_code = 27130 then 90
                        else booked_time_min
                        end as adj_booked_time_min 
                        , wheels_in_to_wheels_out_min
                    from or_intervals 
                    where service != 'Ophthalmology'
                )
            
            , bins as (
                        select 
                            encounter_id
                            , adj_booked_time_min  - wheels_in_to_wheels_out_min as diff
                            , case  
                                when adj_booked_time_min  - wheels_in_to_wheels_out_min < -10 then 'Over 10 min late'
                                when adj_booked_time_min - wheels_in_to_wheels_out_min < -5 then '5-10 min late'
                                when adj_booked_time_min  - wheels_in_to_wheels_out_min < 0 then '0-5 min late'
                                when adj_booked_time_min  - wheels_in_to_wheels_out_min = 0 then 'On time'
                                when adj_booked_time_min - wheels_in_to_wheels_out_min < 5 then '0-5 min early'
                                when adj_booked_time_min  - wheels_in_to_wheels_out_min <= 10 then '5-10 min early'
                                when adj_booked_time_min - wheels_in_to_wheels_out_min > 10 then 'Over 10 min early'
                            end as actual_vs_booked
                        from adj_booked_times
            )
        select 
            actual_vs_booked
            , count(*) as num_surgeries
        from bins
        group by 1
            """))

    actual_vs_booked  num_surgeries
0      0-5 min early            163
1       0-5 min late            269
2     5-10 min early            390
3      5-10 min late            315
4            On time             11
5  Over 10 min early            202
6   Over 10 min late            488


In [114]:
# Question 2. The OR room is cleaned and prepped between services. How long does this take on
# average? Does it take more or less time when the prior service is running late?

print(sqldf("""
            select
                avg(turnaround_time_min)
            from or_intervals
            where service != 'Ophthalmology'
            """))

   avg(turnaround_time_min)
0                 31.707881


In [110]:
# Avg TAT by service
print(sqldf("""
            select
                service
                , avg(turnaround_time_min)
            from or_intervals
            group by 1
            order by 2 desc
            """))


         service  avg(turnaround_time_min)
0        Urology                 34.350649
1       Podiatry                 33.918478
2          OBGYN                 33.000000
3        General                 32.500000
4    Orthopedics                 32.449153
5       Vascular                 31.895522
6            ENT                 30.790850
7        Plastic                 29.875862
8     Pediatrics                 27.000000
9  Ophthalmology                 21.211604


In [166]:
# Do surgeries that end late have longer turnaround times?
print(sqldf("""
            with 
                add_prior_encounter as (
                    select 
                        encounter_id
                        , ended_late
                    from or_intervals 
                    where service != 'Ophthalmology')
        select
            prior_encounter.ended_late
            , avg(current_encounter.turnaround_time_min)
        from or_intervals as current_encounter
        left join add_prior_encounter as prior_encounter
            on current_encounter.prior_encounter_id = prior_encounter.encounter_id
        group by 1
        order by 2 desc
            """))

   ended_late  avg(current_encounter.turnaround_time_min)
0         1.0                                   33.001244
1         0.0                                   31.064516
2         NaN                                   26.312834
